# Access Simulated LSST Alert Data via BigQuery

The Pitt-Google Alert Broker's tabular alert and value-added alert data is stored in [BigQuery](https://docs.cloud.google.com/bigquery/docs/introduction). The dataset and tables names containing simulated LSST data are outlined in our [data listings page](https://mwvgroup.github.io/pittgoogle-client/listings.html#id4).

## Table of Contents

- [Alert tables (overview)](#alert-tables)
- [Value-added tables (overview)](#value-added-tables)
- [SQL query examples](#sql-query-examples)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pittgoogle

# define the Pitt-Google Alert Broker's GCP project name
pgb_project = pittgoogle.ProjectIds().pittgoogle

<a id="alert-tables"></a>
## Alert tables

The alert tables served by the Pitt-Google Alert Broker represent an archive of the `lsst-alerts-simulated` Pub/Sub stream. These tables preserve the structure of the original alert packets (excluding cutouts), including all nested and repeated fields defined in the versioned LSST alert schema. To improve query performance and reduce query costs, these tables are partitioned by the field: `kafkaPublishTimestamp` and clustered by the fields: `healpix9`, `healpix19`, and `healpix29`. 

### Partitions

Partitioning an alert table by the `kafkaPublishTimestamp` field divides the table into daily segments, effectively reducing the number of bytes read by a query when a date (or date range) is specified.

### Clustering

The `healpix9`, `healpix19`, and `healpix29` fields represent the [HEALPix](https://healpix.sourceforge.io) spatial index of the diaSource at three commonly used resolutions ($N_{side} = 2^9$, $2^{19}$, and $2^{29}$). Clustering the alert tables by these fields sorts data within each partition by the HEALPix index, placing nearby sky regions physically close in storage. Depending on the search scale, queries can target coarse (order 9) or fine (order 29) spatial partitions.

The diagram below, obtained from Google's documentation on the [combination of clustered and partitioned tables](https://docs.cloud.google.com/bigquery/docs/clustered-tables#combine-clustered-partitioned-tables) demonstrates how more finely grained sorting is achieved using these methods.

![title](clustering-and-partitioning-tables.png)

The LSST alert schema v9_0 can be accessed [here](https://github.com/lsst/alert_packet/tree/main/python/lsst/alert/packet/schema/9/0) for your convenience. Please note that a newer schema version ([v10_0](https://github.com/lsst/alert_packet/tree/main/python/lsst/alert/packet/schema/10/0)) has recently been made available. You can alternatively explore the schema of the `alerts_v9_0` table in the cell below:

In [2]:
alerts_tbl = pittgoogle.bigquery.Table(name="alerts_v9_0", dataset="lsst_simulated", projectid=pgb_project)
alerts_tbl.schema

,name,type,mode,description
0,diaSourceId,INTEGER,REQUIRED,Identifier of the triggering DiaSource
1,observation_reason,STRING,NULLABLE,Scheduler reason for the image containing this...
2,target_name,STRING,NULLABLE,Scheduler target for the image containing this...
3,diaSource,RECORD,REQUIRED,
4,diaSource.diaSourceId,INTEGER,REQUIRED,Unique identifier of this DiaSource.
...,...,...,...,...
336,MPCORB.t_p,FLOAT,NULLABLE,MJD of pericentric passage [d].
337,healpix9,INTEGER,REQUIRED,HEALPix order 9 pixel index at the source’s ri...
338,healpix19,INTEGER,REQUIRED,HEALPix order 19 pixel index at the source’s r...
339,healpix29,INTEGER,REQUIRED,HEALPix order 29 pixel index at the source’s r...


<a id="value-added-tables"></a>
## Value-added tables

### SuperNNova

Classification results from SuperNNova ([Möller & de Boissière 2019](https://arxiv.org/pdf/1901.06384)). Explore the schema of the `supernnova` table in the cell below:

In [3]:
supernnova_tbl = pittgoogle.bigquery.Table(name="supernnova", dataset="lsst_simulated", projectid=pgb_project)
supernnova_tbl.schema

,name,type,mode,description
0,diaSourceId,INTEGER,REQUIRED,Unique identifier of this DiaSource.
1,diaObjectId,INTEGER,NULLABLE,Unique identifier of this DiaObject.
2,ssObjectId,INTEGER,NULLABLE,Unique identifier of the object.
3,prob_class0,FLOAT,NULLABLE,probability object is a type Ia supernova
4,prob_class1,FLOAT,NULLABLE,probability object is non-Ia
5,predicted_class,INTEGER,NULLABLE,"predicted class; 0 = Ia, 1 = non-Ia"
6,kafkaPublishTimestamp,TIMESTAMP,REQUIRED,Kafka timestamp from originating LSST alert.


### UPSILoN

Classification results from UPSILoN ([Kim & Bailer-Jones 2015](https://arxiv.org/pdf/1512.01611)). Explore the schema of the `upsilon` table in the cell below:

In [4]:
upsilon_tbl = pittgoogle.bigquery.Table(name="upsilon", dataset="lsst_simulated", projectid=pgb_project)
upsilon_tbl.schema

,name,type,mode,description
0,diaSourceId,INTEGER,REQUIRED,Unique identifier of this DiaSource.
1,diaObjectId,INTEGER,NULLABLE,Unique identifier of this DiaObject.
2,ssObjectId,INTEGER,NULLABLE,Unique identifier of the object.
3,u_label,STRING,NULLABLE,Predicted class using u band photometry.
4,g_label,STRING,NULLABLE,Predicted class using g band photometry.
5,r_label,STRING,NULLABLE,Predicted class using r band photometry.
6,i_label,STRING,NULLABLE,Predicted class using i band photometry.
7,z_label,STRING,NULLABLE,Predicted class using z band photometry.
8,y_label,STRING,NULLABLE,Predicted class using y band photometry.
9,u_probability,FLOAT,NULLABLE,Class probability for u_label.


### Variability

The following table contains Stetson's J variability index ([Stetson 1996](https://iopscience.iop.org/article/10.1086/133808/pdf)). Explore the schema of the `variability` table in the cell below:

In [5]:
variability_tbl = pittgoogle.bigquery.Table(name="variability", dataset="lsst_simulated", projectid=pgb_project)
variability_tbl.schema

,name,type,mode,description
0,diaSourceId,INTEGER,REQUIRED,Unique identifier of this DiaSource.
1,diaObjectId,INTEGER,NULLABLE,Unique identifier of this DiaObject.
2,ssObjectId,INTEGER,NULLABLE,Unique identifier of the object.
3,u_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the u band DI...
4,g_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the g band DI...
5,r_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the r band DI...
6,i_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the i band DI...
7,z_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the z band DI...
8,y_psfFluxStetsonJ,FLOAT,NULLABLE,Stetson J index calculated using the y band DI...
9,n_detections_u_band,INTEGER,NULLABLE,Number of detections of this object in the u b...


<a id="sql-query-examples"></a>
## SQL query examples

[SQL queries](https://www.geeksforgeeks.org/sql/what-is-sql/) can be used to access the data stored in the BigQuery tables served by the Pitt-Google Alert Broker. By default the first 1 TiB of query data processed per month is free for every Google Cloud Platform user. See [here](https://mwvgroup.github.io/pittgoogle-client/faq/cost.html) for more information on costs.

In [6]:
# the query below returns the distinct date values in the kafkaPublishTimestamp field of the alerts table
alerts_tbl.query(columns=["DISTINCT(DATE(kafkaPublishTimestamp))"])

,f0_
0,2025-09-03


As we see in the cell above, the `alerts_v9_0` only contains data published on 2025-09-03. When LSST begins, it will be useful to construct queries containing date ranges. An example is outlined below:

In [7]:
column = ["DISTINCT(diaObject.diaObjectId)"]
where = 'DATE(kafkaPublishTimestamp) BETWEEN "2025-09-01" AND "2025-09-05"'
alerts_tbl.query(columns=column, where=where)

,diaObjectId
0,396950876717059
1,397017482264581
2,396841556377630
3,396841724149795
4,396950608281767
...,...
617,396841556377610
618,396895612567643
619,396841690595429
620,396895243468862


In [8]:
# define query parameters
column = ["diaSource.psfFlux, diaSource.psfFluxErr, diaSource.midpointMjdTai, diaSource.band"]
where = 'DATE(kafkaPublishTimestamp) = "2025-09-03" AND diaObject.diaObjectId = 396841556377652'

# execute query
photometry = alerts_tbl.query(columns=column, where=where)
photometry

,psfFlux,psfFluxErr,midpointMjdTai,band
0,-350.492126,89.527313,57454.309943,g
1,-179.487610,126.140945,57454.299200,g
2,-882.617126,90.736061,57454.329282,g
3,377.658020,70.898506,57454.289074,g


An alternative method of writing and executing SQL queries can be done using the BigQuery client. This is the appropriate choice to make when writing more complicated queries (e.g., JOIN statements).

In [10]:
bqclient = pittgoogle.bigquery.Client() # load the BigQuery client

sql_query = f"""
SELECT
  alerts.diaSourceId,
  alerts.diaSource.psfFlux AS psfFlux,
  alerts.diaSource.psfFluxErr AS psfFluxErr,
  alerts.diaObject.diaObjectId,
  var.g_psfFluxStetsonJ
FROM `{pgb_project}.lsst_simulated.alerts_v9_0` AS alerts
INNER JOIN `{pgb_project}.lsst_simulated.variability` AS var
  ON alerts.diaObject.diaObjectId = var.diaObjectId
WHERE var.g_psfFluxStetsonJ > 2
  AND var.n_detections_g_band > 3
"""

bqclient.query(query=sql_query)

,diaSourceId,psfFlux,psfFluxErr,diaObjectId,g_psfFluxStetsonJ
0,397017750700097,2259.302490,44.843967,396895243468892,4.584567
1,396895411241042,-273.139221,38.426113,396895411241042,17.988549
2,397017750700102,-407.678101,45.858158,396841690595425,2.238800
3,396841556377619,385.961823,85.030296,396841556377619,6.079313
4,396950876717098,-350.492126,89.527313,396841556377652,2.213091
...,...,...,...,...,...
78,396841556377652,377.658020,70.898506,396841556377652,2.213091
79,396841556377652,377.658020,70.898506,396841556377652,4.826614
80,397017750700071,-50.287994,53.666119,396841556377656,9.580622
81,396950876717182,-1427.866333,33.048271,396841724150041,7.699976
